In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pyodbc
import warnings

warnings.filterwarnings("ignore")

## Users With Reg Date
TOTOID, UserID, Registration Date, Age

In [2]:
indexes = pd.read_csv('model_data.csv', index_col='TOTOID')
indexes

,DepositCount,DepositAmount,Age,Category,One_game,Second_game
TOTOID,,,,,,
100377572,21,5200.0,24,Slots,True,Betongames
100377577,96,274170.0,57,Sports,True,Betongames
100377612,33,248372.0,45,P2P,True,Slots
100377613,143,443500.0,59,Slots,True,Betongames
100377617,500,2797000.0,42,Slots,True,NaN
...,...,...,...,...,...,...
101242827,2,3000.0,26,Live Casino,True,Slots
101242831,1,10000.0,64,Slots,True,Betongames
101242837,1,1000.0,37,Slots,False,Betongames


In [3]:
list_of_users = str(tuple(indexes.index))
list_of_users = list_of_users.replace('(', '')
list_of_users = list_of_users.replace(')', '')

In [6]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
SELECT cu.PartnerUserID                                      AS TOTOID,
       cu.UserID                                             AS UserID,
       DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE)) AS Age,
       IIF(CAST(cu.RegistrationDate AS DATE) > CAST(su.RegistrationDate AS DATE), CAST(su.RegistrationDate AS DATE),
           CAST(cu.RegistrationDate AS DATE))                AS RegDate
FROM VIEW_PlatformPartnerUsers_TotogamingAm cu
    LEFT JOIN VIEW_sport_PartnerUser_TotogamingAm su ON cu.PartnerUserId = su.PartnerUserId
WHERE DATEDIFF(DAY, CAST(su.RegistrationDate AS DATE), CAST(cu.RegistrationDate AS DATE)) <= 5
  AND DATEDIFF(DAY, CAST(su.RegistrationDate AS DATE), CAST(cu.RegistrationDate AS DATE)) >= -5
GROUP BY cu.PartnerUserID, cu.UserID, cu.RegistrationDate, su.RegistrationDate, DATEDIFF(YEAR, cu.BirthDate, CAST(GETDATE() AS DATE))
"""
users = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

In [7]:
users

,UserID,Age,RegDate
TOTOID,,,
100382268,2515062,56,2016-02-06
100388859,2544045,30,2017-03-13
100415225,2570323,65,2017-10-20
100476999,2639932,31,2018-03-11
100516220,4395801,26,2018-06-10
...,...,...,...
100617717,6417443,28,2019-02-07
100480467,2643602,82,2018-03-18
101050958,15584515,35,2021-01-26


## User first 6 months deposits

In [ ]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
SELECT a.TOTOID          AS TOTOID,
    SUM(a.DepositCount)  AS DepositCount,
    SUM(a.DepositAmount) AS DepositAmount,
    a.EndDate            AS EndDate
FROM
(SELECT cu.PartnerUserID                                               AS TOTOID,
    COUNT(p.PaymentID)                                                 AS DepositCount,
    SUM(p.Amount)                                                      AS DepositAmount,
    CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE) AS EndDate
FROM Payment p
INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
	AND p.PaymentStatusID = 8
    AND p.PaymentTypeID = 2
    AND p.SourceID = 2 --Casino Source
    AND sp.PaymentSystemName NOT IN ('PokerTransfer','TRANSFER')
    AND cu.PartnerUserId IN ({list_of_users})
GROUP BY cu.PartnerUserID, cu.RegistrationDate

UNION ALL

SELECT cu.PartnerUserID                                                AS TOTOID,
    COUNT(p.PaymentID)                                                 AS DepositCount,
    SUM(p.Amount)                                                      AS DepositAmount,
    CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE) AS EndDate
    FROM Payment p
INNER JOIN VIEW_sport_PartnerUser_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
    AND p.TransactionTypeID IN (62,65,70,73,87,88,102,103,75,71)
    AND p.Fund_StatusID IN (3,23,31,53,63,106)
    AND p.SourceID = 1  --Sport Source
    AND sp.PaymentSystemName NOT IN ('Casino Transfer','BONUS')
    AND cu.PartnerUserId IN ({list_of_users})
GROUP BY cu.PartnerUserID, cu.RegistrationDate) a
GROUP BY a.TOTOID, a.EndDate
ORDER BY TOTOID
"""
deposits = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

In [ ]:
deposits

## Bets with Games

In [ ]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
    DROP TABLE IF EXISTS #cg
    SELECT cg.GameID,
        cg.GameCategoryID,
        cg.GameProviderID,
        cg.Name_en
    INTO #cg
    FROM C_Game cg

    DROP TABLE IF EXISTS #gc
    SELECT gc.GameCategoryID,
        gc.GameCategoryName
    INTO #gc
    FROM C_GameCategory gc

    DROP TABLE IF EXISTS #gp
    SELECT gp.GameProviderID,
        gp.GameProviderName
    INTO #gp
    FROM C_GameProvider gp

    DROP TABLE IF EXISTS #a
    SELECT #cg.GameID,
        #cg.Name_en,
        #gc.GameCategoryName,
        #gp.GameProviderName
    INTO #a
    FROM #cg
    LEFT JOIN #gc ON #cg.GameCategoryID = #gc.GameCategoryID
    LEFT JOIN #gp ON #cg.GameProviderID = #gp.GameProviderID

    DROP TABLE IF EXISTS #cas
    SELECT u.PartnerUserID,
        count(*) AS BetCount,
        SUM(a.BetAmount) AS BetAmount,
        g.GameID
    INTO #cas
    FROM (
    SELECT o.UserID,
        o.GameID,
        (IIF(o.CalculationDate < '2021-03-01', o.OrderAmount,
        CASE WHEN cg.GameProviderID IN (48, 10) AND o.TypeId IN (1, 5, 8, 18, 33) THEN o.OrderAmount
        WHEN cg.GameProviderID NOT IN (48, 10) THEN o.OrderAmount ELSE 0 END)) AS BetAmount
    FROM casino.orders o
    INNER JOIN C_Game cg ON cg.GameID = o.GameID
    INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON o.UserID = u.UserID
    INNER JOIN C_GameProvider gp ON gp.GameProviderID = cg.GameProviderID
    WHERE o.CalculationDate <= CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date)
    AND u.PartnerUserId IN ({list_of_users})
    AND gp.GameProviderID <> 3
    AND OrderStateID IN (2, 3, 5, 6)
    AND o.OperationTypeID IN (3, 299)
    AND IIF(cg.GameProviderID IN (48, 10)
    AND o.CalculationDate < '2021-03-01', o.TypeId, 0) IN (0, 1, 5, 8, 18, 33)
    ) a
    INNER JOIN C_Game g ON g.GameID = a.GameID
    INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON u.UserID = a.UserID
    WHERE u.isDeleted = 0
    AND u.UserTypeID NOT IN (1, 20, 3, 21)
    AND u.UserName NOT LIKE 'test%'
    AND u.UserName NOT LIKE '%TestClient%'
    GROUP BY u.PartnerUserID,
        g.GameID

    SELECT * FROM (SELECT #cas.PartnerUserId AS TOTOID,
        SUM(#cas.BetCount) AS Bet_count,
        SUM(#cas.BetAmount) AS Bet_amount,
        #b.Type AS Category
    FROM #cas
    LEFT JOIN (SELECT *, (
    CASE WHEN #a.Name_en LIKE '%HabaneroJackpot%'
        THEN 'Other'
    WHEN #a.GameProviderName LIKE '%Kiron%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Greyhound Racing%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Penalty Shootout%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Flat Horse Racing%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Fantastic League%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%Horse Racing%'
        THEN 'Virtual Games'
    WHEN #a.Name_en LIKE '%English league%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%Digitain%' AND #a.Name_en LIKE '%Football Single Match%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%GlobalBet%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%GoldenRace%' AND #a.Name_en in ('Keno','Keno Deluxe')
        THEN 'TVGames'
    WHEN #a.GameProviderName LIKE '%Golden%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%EventBet%'
        THEN 'Poker'
    WHEN #a.GameProviderName LIKE '%Betgames%'
        THEN 'TVGames'
    WHEN #a.GameProviderName LIKE '%Atmosfera%' AND #a.Name_en in ('Bingo 37','Keno','Bingo 38')
        THEN 'TVGames'
    WHEN #a.GameProviderName = 'FLG' AND #a.Name_en in ('Keno Gold')
        THEN 'TVGames'
    WHEN #a.GameProviderName LIKE '%Betongames%'
        THEN 'Betongames'
    WHEN #a.GameProviderName LIKE '%Evolution%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%LiveCasino%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%EzugiOriginal%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%LuckyStreak%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%VivoGaming%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%NetEntLiveCasino%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%TV Bet%'
        THEN 'TVGames'
    WHEN #a.GameProviderName LIKE '%BetGames%'
        THEN 'TVGames'
    WHEN #a.GameCategoryName LIKE '%P2P%'
        THEN 'P2P'
    WHEN #a.GameProviderName LIKE '%GGPoker%'
        THEN 'Poker'
    WHEN #a.GameCategoryName LIKE '%Slot%'
        THEN 'Slots'
    WHEN #a.GameProviderName LIKE '%InBet%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%Leap%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%Digitain Live Casino%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Digitain%'
        THEN 'Sport'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%Baccarat%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%Blackjack%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%TV Show%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.GameCategoryName LIKE '%GameShow%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.Name_en LIKE '%Mega Sic Bo%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.Name_en LIKE '%Dragon Tiger%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Pragmatic%' AND #a.Name_en LIKE '%Andar Bahar%'
        THEN 'Live Casino'
    WHEN #a.Name_en LIKE '%Dynamic Roulette 120x%'
        THEN 'Live Casino'
    WHEN #a.Name_en LIKE '%Live European Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Atmosfera%' AND #a.Name_en LIKE '%Live Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Atmosfera%' AND #a.Name_en LIKE '%Music Wheel%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%Atmosfera%' AND #a.Name_en LIKE '%Auto Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%XPG%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%Onyx Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%Live Speed Roulette%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%Vegas Roulette 500x%'
        THEN 'Live Casino'
    WHEN #a.GameProviderName LIKE '%PlayngoArmenia%' AND #a.Name_en LIKE '%European Roulette Pro%'
        THEN 'Slots'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%Burning Keno Plus%'
        THEN 'Slots'
    WHEN #a.GameProviderName LIKE '%1x2%' AND #a.Name_en LIKE '%Instant Football%'
        THEN 'Virtual Games'
    WHEN #a.GameProviderName LIKE '%EGT%' AND #a.Name_en LIKE '%European Roulette%'
        THEN 'Slots'
    ELSE 'Other'
    END) Type
    FROM #a ) #b ON #cas.GameID = #b.GameID
    GROUP BY #cas.PartnerUserId, #b.Type

    UNION ALL

    SELECT u.PartnerUserID AS TOTOID,
        COUNT(o.OrderID) AS Order_count,
        SUM(o.OrderAmount) AS Order_amount,
        'Sports' AS Category
    FROM VIEW_sport_PartnerUser_TotogamingAm u
    INNER JOIN VIEW_sport_Orders_TotogamingAm o ON o.UserID = u.UserID
    WHERE o.OrderDate_DT <= CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date)
    AND u.PartnerID IN ({list_of_users})
    AND o.OrderStateID IN (2, 3, 5, 6)
    AND u.UserTypeID NOT IN (1, 20, 3, 21)
    AND u.UserName NOT LIKE 'test%'
    AND u.UserName NOT LIKE '%TestClient%'
    GROUP BY u.PartnerUserId) AS bc
    ORDER BY bc.TOTOID
    """
query = 'SET NOCOUNT ON; \n' + query
bets_games = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

In [ ]:
bets_games

## Bet History

In [3]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
DROP TABLE IF EXISTS #cas
SELECT u.PartnerUserID,
       count(*) as BetCount,
       SUM(a.BetAmount) AS BetAmount,
       a.Date,
       a.EndDate
INTO #cas
FROM (SELECT o.UserID,
             o.GameID,
             (IIF(o.CalculationDate < '2021-03-01', o.OrderAmount,
                  CASE
                      WHEN cg.GameProviderID IN (48, 10) AND o.TypeId IN (1, 5, 8, 18, 33) THEN o.OrderAmount
                      WHEN cg.GameProviderID NOT IN (48, 10) THEN o.OrderAmount
                      ELSE 0 END))           AS BetAmount,
             o.CalculationDate_DT AS Date,
             CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date) AS EndDate
      FROM casino.orders o
               INNER JOIN C_Game cg ON cg.GameID = o.GameID
               INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON o.UserID = u.UserID
               INNER JOIN C_GameProvider gp ON gp.GameProviderID = cg.GameProviderID
      WHERE o.CalculationDate <= CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date)
        AND gp.GameProviderID <> 3
        AND u.PartnerUserId IN ({list_of_users})
        AND OrderStateID IN (2, 3, 5, 6)
        AND o.OperationTypeID IN (3, 299)
        AND IIF(cg.GameProviderID IN (48, 10)
                    AND o.CalculationDate < '2021-03-01', o.TypeId, 0) IN (0, 1, 5, 8, 18, 33)) a
INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm u ON u.UserID = a.UserID
WHERE u.isDeleted = 0
  AND u.UserTypeID NOT IN (1, 20, 3, 21)
  AND u.UserName NOT LIKE 'test%'
  AND u.UserName NOT LIKE '%TestClient%'
GROUP BY u.PartnerUserID,
         a.Date

SELECT * FROM (
SELECT #cas.PartnerUserId  AS TOTOID,
       #cas.Date           AS Date,
       SUM(#cas.BetCount)  AS Bet_count,
       SUM(#cas.BetAmount) AS Bet_amount,
       #cas.EndDate        AS EndDate
FROM #cas
GROUP BY #cas.PartnerUserId, #cas.Date, #cas.EndDate

UNION ALL

SELECT u.PartnerUserID               AS TOTOID,
       o.OrderDate_DT                AS DATE ,
       COUNT(o.OrderID)              AS Bet_count,
       SUM(o.OrderAmount)            AS Bet_amount,
       CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date) AS EndDate
FROM VIEW_sport_PartnerUser_TotogamingAm u
         INNER JOIN VIEW_sport_Orders_TotogamingAm o ON o.UserID = u.UserID
WHERE o.OrderDate_DT <= CAST(DATEADD(MONTH, 6, CAST(u.RegistrationDate AS DATE)) AS Date)
  AND o.OrderStateID IN (2, 3, 5, 6)
  AND u.PartnerUserId IN ({list_of_users})
  AND u.UserTypeID NOT IN (1, 20, 3, 21)
  AND u.UserName NOT LIKE 'test%'
  AND u.UserName NOT LIKE '%TestClient%'
GROUP BY u.PartnerUserId, o.OrderDate_DT, u.RegistrationDate) AS bc
ORDER BY bc.TOTOID
"""
query = 'SET NOCOUNT ON; \n' + query
bets = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

NameError: name 'list_of_users' is not defined

In [ ]:
bets

## Deposits History

In [ ]:
cnx: pyodbc.Connection = pyodbc.connect(
    "Driver={ODBC Driver 17 for SQL Server};"
    "Server=DWH;"
    "Database=dwOper;"
    "Trusted_Connection=yes;")
query = f"""
SELECT a.TOTOID,
    SUM(a.DepositCount) AS DepositCount,
    SUM(a.DepositAmount) AS DepositAmount,
    a.DepositDate AS Date
FROM
(SELECT cu.PartnerUserID AS TOTOID,
    COUNT(p.PaymentID) DepositCount,
    SUM(p.Amount) DepositAmount,
    CAST(p.modify_date AS DATE) AS DepositDate
FROM Payment p
INNER JOIN VIEW_PlatformPartnerUsers_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
    AND cu.PartnerUserID IN ({list_of_users})
	AND p.PaymentStatusID = 8
    AND p.PaymentTypeID = 2
    AND p.SourceID = 2 --Casino Source
    AND sp.PaymentSystemName NOT IN ('PokerTransfer','TRANSFER')
GROUP BY cu.PartnerUserID, CAST(p.modify_date AS Date)

UNION

SELECT cu.PartnerUserID AS TOTOID,
    COUNT(p.PaymentID) DepositCount,
    SUM(p.Amount) DepositAmount,
    CAST(p.modify_date AS Date) AS DepositDate
FROM Payment p
INNER JOIN VIEW_sport_PartnerUser_TotogamingAm cu ON p.UserID = cu.UserID
LEFT JOIN C_PaymentSystem sp ON sp.PaymentSystemId = p.PaymentSystemID
WHERE CAST(p.modify_date AS DATE) <= CAST(DATEADD(MONTH, 6, CAST(cu.RegistrationDate AS DATE)) AS DATE)
	AND cu.PartnerUserID IN ({list_of_users})
    AND p.TransactionTypeID IN (62,65,70,73,87,88,102,103,75,71)
    AND p.Fund_StatusID IN (3,23,31,53,63,106)
    AND p.SourceID = 1  --Sport Source
    AND sp.PaymentSystemName NOT IN ('Casino Transfer','BONUS')
GROUP BY cu.PartnerUserID, CAST(p.modify_date AS Date)) a
GROUP BY a.TOTOID, a.DepositDate
ORDER BY TOTOID
"""
query = 'SET NOCOUNT ON; \n' + query
deposit_hist = pd.read_sql(query, cnx, index_col='TOTOID')
cnx.close()

In [ ]:
deposit_hist

In [ ]:
def get_game_attributes(df):
    category_arr = []
    for i in df.index.unique():
        temp = df[df.index == i]
        if len(temp) == 1:
            category = temp.iloc[0, 2]
            category_set = {'TOTOID': i, 'Category': category, 'One_game': True, 'Second_game': None}
            category_arr.append(category_set)
        else:
            largest_amount = temp['Bet_amount'].nlargest(1).values[0]
            second_largest = temp['Bet_amount'].nlargest(2).values[1]
            thresh = sum(temp['Bet_amount']) * 0.7
            fav = temp.query(f"`Bet_amount` == {largest_amount}").iloc[0, 2]
            second_fav = temp.query(f"`Bet_amount` == {second_largest}").iloc[0, 2]
            if largest_amount >= thresh:
                category_set = {'TOTOID': i, 'Category': fav, 'One_game': True, 'Second_game': second_fav}
                category_arr.append(category_set)
            else:
                category_set = {'TOTOID': i, 'Category': fav, 'One_game': False, 'Second_game': second_fav}
                category_arr.append(category_set)
    df_ = pd.DataFrame.from_dict(category_arr)
    df_.set_index('TOTOID', inplace=True)
    return df_

In [ ]:
games = get_game_attributes(bets_games)

In [ ]:
games

In [ ]:
users = users.merge(games, on='TOTOID')

In [ ]:
bets.index.unique()

In [ ]:
arr = []
for i in bets.index.unique():
        values = []
        temp = bets[bets.index == i].sort_values('Date')
        for l in range(len(temp) - 1):
            date_1 = temp.iloc[l, 0]
            date_2 = temp.iloc[l+1, 0]
            delta = date_2 - date_1
            values.append(delta.days)
        arr.append({'TOTOID':i, 'AVGLag':np.mean(values)})
arr

In [ ]:
bet_lag = pd.DataFrame.from_dict(arr)

In [ ]:
bet_lag

In [ ]:
arr = []
for i in deposit_hist.index.unique():
        values = []
        temp = deposit_hist[deposit_hist.index == i].sort_values('Date')
        for l in range(len(temp) - 1):
            date_1 = temp.iloc[l, 2]
            date_2 = temp.iloc[l+1, 2]
            delta = date_2 - date_1
            values.append(delta.days)
        arr.append({'TOTOID':i, 'AVGLag':np.mean(values)})
arr

In [ ]:
deposit_lag = pd.DataFrame.from_dict(arr)

In [ ]:
deposit_lag